In [62]:
import timeit # https://www.geeksforgeeks.org/timeit-python-examples/
SETUP_CODE = ''' 

class Heap:
    """sources consulted: https://www.geeksforgeeks.org/leftist-tree-leftist-heap/"""
    __slots__ = "_right", "_left", "_rank", "_key", "_othervalues"

    def __init__(self, key, othervalues):
        self._left = None
        self._right = None
        self._rank = 1
        self._key = key
        self._othervalues = othervalues
        
    def findmin(self):
        return self
    
    def meld(h1, h2):
        """ merges the heaps h1 and h2 and returns a """
        if h1 == None:
            return h2
        else:
            if h2 == None:
                return h1
            else: # h1 not None and h2 not None
                if h1._key > h2._key:
                    h1, h2 = h2, h1
                return Heap.mesh(h1,h2)

    
    def mesh(h1,h2):
        """ subroutine in meld """
        if h1._left == None:
            h1._left = h2
        else: # h1._right not None
            h1._right = Heap.meld(h1._right,h2)
            
            if h1._left == None:  # make None less than any h1._right, since Python 3 does not compaire None and int
                h1._left, h1._right = h1._right, h1._left
            elif h2._right is not None and h1._left._rank < h1._right._rank: # make None less than any h1._right 
                h1._left, h1._right = h1._right, h1._left

        h1._rank = h1._rank + 1
        return h1
        
    def insert(self, key, othervalues):
        hnew = Heap(key,othervalues)
        self = Heap.meld(self, hnew)
        
    def deletemin(self):
        self = Heap.meld(self._left,self._right)
        return self

    
    def printout(self):
        print("entering printout")
        print(self._key, self._othervalues)
        if self._left is not None:
            print("left")
            self._left.printout()
        else: 
            print("self._left = None")
        if self._right is not None:
            print("right")
            self._right.printout()
        else:
            print("self._right = None")
            
class Partition:
    """Union-find structure for maintaining disjoint sets."""
    """code from the textbook: https://doc.lagout.org/programmation/python/Data%20Structures%20and%20Algorithms%20in%20Python%20[Goodrich,%20Tamassia%20&%20Goldwasser%202013-03-18].pdf"""

    #------------------------- nested Position class -------------------------
    class Position:
        __slots__ = '_container' , '_element' , '_size' , '_parent'
        
        def __init__(self, container, e):
            """Create a new position that is the leader of its own group."""
            self._container = container # reference to Partition instance
            self._element = e
            self._size = 1
            self._parent = self # convention for a group leader

        def element(self):
            """Return element stored at this position."""
            return self._element

    #------------------------- public Partition methods -------------------------
    def make_group(self, e):
        """Makes a new group containing element e, and returns its Position"""
        return self.Position(self, e)
    
    def find(self, p):
        """Finds the group containging p and return the position of its leader."""
        if p._parent != p:
            p._parent = self.find(p._parent) # overwrite p. parent after recursion
        return p._parent
    
    def union(self, p, q):
        """Merges the groups containg elements p and q (if distinct)."""
        a = self.find(p)
        b = self.find(q)
        if a is not b: # only merge if different groups
            if a._size > b._size:
                b._parent = a
                a._size += b._size
                return a # I added to keep track of how many groups
            else:
                a._parent = b
                b._size += a._size
                return b # I added to keep track of how many groups
                
# Sollins Algorithm
# read in the graph
# file = open("mst5.txt","r") # not working, search for ugroup_index, vgroup_index 0 1
# file = open("mst9.txt","r")
#file = open("mst100_200.txt","r")
#file = open("mst100_4000.txt","r")
#file = open("mst500_2000.txt","r")
#file = open("mst500_4000.txt","r")
#file = open("mst500_6000.txt","r")
#file = open("mst500_8000.txt","r")
#file = open("mst500_10000.txt","r")
#file = open("mst500_12000.txt","r")
#file = open("mst1000_2000.txt","r")
#file = open("mst1000_4000.txt","r")
#file = open("mst1000_6000.txt","r")
#file = open("mst1000_8000.txt","r")
#file = open("mst1000_10000.txt","r")
#file = open("mst1000_12000.txt","r")

#file = open("mst2000_12000.txt","r")
#file = open("mst2000_14000.txt","r")
#file = open("mst2000_20000.txt","r")
#file = open("mst2000_30000.txt","r")
#file = open("mst2000_40000.txt","r")
#file = open("mst2000_50000.txt","r")

#file = open("mst5000_12000.txt","r")
file = open("mst10000_50000.txt","r")

#file = open("mst15.txt","r")
# file = open("distconected1.txt","r")
for i in range(2):
    file.readline()
infoline = file.readline()
file.readline() # extra line
paramline = file.readline()
linelist = paramline.split()
n_nodes = int(linelist[2])
n_arcs = int(linelist[3])

# setup the data structures
P = Partition() # a disjoint set
Grouplist = [] # a list to hold the Groups of the Partition
Vertexlist = [] # a non changing list of the pointers to the Position objects for each vertex
Group_to_Grouplistindex = {} # a dictionary with Group objects as keys and the indices as values
                                # the indices correspond to where the Group is in the Grouplist
Heaplist = [] # a list to hold the Heaps corresponding to each Group in the Grouplist

numGroups = n_nodes
for i in range(0,n_nodes):
    Grouplist.append(P.make_group(i))
    Vertexlist.append(Grouplist[i])
    Group_to_Grouplistindex[Grouplist[i]] = i
    Heaplist.append(None)

for  line in file: # loop through edges
    linelist = line.split()
    u = int(linelist[1])
    v = int(linelist[2])
    w = int(linelist[3])
    if u > v:
        u , v = v , u 
#     print(w,(u,v))
    H = Heap(w, (u,v))
    Heaplist[u] = Heap.meld(H,Heaplist[u])
    H = Heap(w, (u,v))
    Heaplist[v] = Heap.meld(H,Heaplist[v])
file.close()
'''

TEST_CODE = ''' 
mst = []
totalloops = 0
disconectedGraph = False 
while numGroups > 1:
    if totalloops > n_arcs :
        print("###############infinite loop detected")
        disconectedGraph = True
        break
    totalloops = totalloops + 1
#     print("totalloops",totalloops)
    i = 0
    
    while i < numGroups:
        
#         print("The Groups when numGroups, i = ", numGroups,i)
        
#         print("There are the Heaps when numGroups, i = ", numGroups, i)
#         for j in range(0,numGroups):
#             print("*********Heap  ",j ,"  *************")
#             if Heaplist[j] == None:
#                 print("None")
#             else:
#                 Heaplist[j].printout()

#         for kk in range(numGroups):
#             index = Group_to_Grouplistindex[P.find(Grouplist[kk])]
#             if index != kk:
#                 print("index",index)
#                 for j in range(0,numGroups):
#                     print("*********Heap  ",j ,"  *************")
#                     if Heaplist[j] == None:
#                         print("None")
#                     else:
#                         Heaplist[j].printout()
#             print(Group_to_Grouplistindex[P.find(Grouplist[kk])])
        if Heaplist[i] == None:
            disconectedGraph = True
            break
        edge_weight , (u,v) = Heaplist[i]._key, Heaplist[i]._othervalues
        Heaplist[i] = Heaplist[i].deletemin()
        ugroup = P.find(Vertexlist[u])
        vgroup = P.find(Vertexlist[v])
        ugroup_index = Group_to_Grouplistindex[ugroup]
        vgroup_index = Group_to_Grouplistindex[vgroup]

        if ugroup_index == vgroup_index:
            pass # don't add this edge to the mst
        else:
            mst.append(((u,v),edge_weight))
            if ugroup_index < vgroup_index:
                Grouplist[ugroup_index] = P.union(ugroup,vgroup)
                Heaplist_ugroup_index = Heaplist[ugroup_index] # to avoid a swap in the pointers in Heaplist
                Heaplist_vgroup_index = Heaplist[vgroup_index]
                Heaplist[ugroup_index] = Heap.meld(Heaplist_ugroup_index, Heaplist_vgroup_index)
                Group_to_Grouplistindex[vgroup] = ugroup_index #update where this group is now
                if vgroup_index != numGroups -1:
                    Grouplist[vgroup_index], Heaplist[vgroup_index] = Grouplist[numGroups-1], Heaplist[numGroups-1]
                    Group_to_Grouplistindex[Grouplist[numGroups-1]] = vgroup_index # update where this group is now
                Grouplist[numGroups-1], Heaplist[numGroups-1] = None, None  # this line is needed, mst6.txt fails otherwis
                numGroups = numGroups - 1
            else: # ugroup_index > vgroup_index
                Grouplist[vgroup_index] = P.union(ugroup,vgroup)
                Heaplist_ugroup_index = Heaplist[ugroup_index]
                Heaplist_vgroup_index = Heaplist[vgroup_index]
                Heaplist[vgroup_index] = Heap.meld(Heaplist_vgroup_index,Heaplist_ugroup_index)
                Group_to_Grouplistindex[ugroup] = vgroup_index #update where this group is now
                if ugroup_index != numGroups -1:
                    Grouplist[ugroup_index], Heaplist[ugroup_index] = Grouplist[numGroups-1], Heaplist[numGroups-1]
                    Group_to_Grouplistindex[Grouplist[numGroups-1]] = ugroup_index # update where this group is now
                Grouplist[numGroups-1], Heaplist[numGroups-1] = None, None
                numGroups = numGroups - 1
        i = i + 1
        
'''

times = timeit.repeat(setup = SETUP_CODE, 
                          stmt = TEST_CODE, 
                          repeat = 3, 
                          number = 10000) 
  
    # priniting minimum exec. time
print('min time: {}'.format(min(times)))
print('average time: {}'.format(sum(times)/len(times)))



# if disconectedGraph:
#     print("The graph is disconnected, the output is a spanning forest")
    
# # print("number of arcs in mst", len(mst)) 
# n_mstarcs = len(mst)
# print("DIMACS Output\n")
# print("c Minimum Spanning Tree")
# print("c")
# infolinelist = infoline.split(';')
# infolinelist[1] = "arcs:  " + str(n_mstarcs)
# # print(infolinelist)
# infolinenew = infolinelist[0] + "; " + infolinelist[1] + " ;" + infolinelist[2]
# print(infolinenew, end = "")
# print("c")
# print(paramline, end="")
# for edge in mst:
# #     print(edge)
#     print("e", edge[0][0], edge[0][1], edge[1])  

min time: 0.13010099300026923
average time: 0.13112616633346383
